In [1]:
%load_ext autoreload
%autoreload 2


In [1]:
# import sys
# import os

# # Print debugging information
# print(f"Python path: {sys.executable}")
# print(f"Python version: {sys.version}")
# print(f"sys.path: {sys.path}")

# try:
#     from langchain_openai import ChatOpenAI
#     print("Successfully imported ChatOpenAI")
# except ImportError as e:
#     print(f"Import error: {e}")
#     print("Try running: pip install langchain-openai")
#     sys.exit(1)

# # Initialize the ChatOpenAI instance with your API key
# llm = ChatOpenAI(
#     api_key=os.environ.get("OPENAI_API_KEY")
# )

# # Test the chat
# response = llm.invoke("Hello, world!")
# print(response)

Python path: /Users/matthew.khoriaty/opt/anaconda3/envs/langchain/bin/python
Python version: 3.11.11 (main, Dec 11 2024, 10:25:04) [Clang 14.0.6 ]
sys.path: ['/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python311.zip', '/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11', '/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/lib-dynload', '', '/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/site-packages']
Successfully imported ChatOpenAI
content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': Non

In [17]:
# Convert Dataset to pandas DataFrame to allow item assignment
# df = df.to_pandas()
def make_dataset_csv():
    import pandas as pd

    # splits = {'dev': 'data/dev-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet', 'train': 'data/train-00000-of-00001.parquet'}
    # df = pd.read_parquet("hf://datasets/princeton-nlp/SWE-bench/" + splits["dev"])
    from datasets import load_dataset

    ds = load_dataset("princeton-nlp/SWE-bench")
    df = ds['dev']
    df['version'] = df['version'].apply(lambda x: f"version:{x}")
    df.to_csv("SWE-bench.csv",index=False)

In [3]:
from langsmith import evaluate
from langsmith import Client

client = Client()

def predict(inputs: dict):
    return {"instance_id":inputs['instance_id'],"model_patch":"None","model_name_or_path":"test-model"}


In [4]:

result = evaluate(
    predict,
    data=client.list_examples(dataset_id="58c8677a-00fa-47c2-89b4-941cb09a47be"),
)

/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'crazy-grass-94' at:
https://smith.langchain.com/o/22d9505f-5add-45d4-84e0-97f2aa494e0d/datasets/58c8677a-00fa-47c2-89b4-941cb09a47be/compare?selectedSessions=33849ff4-eb5d-4f87-a574-c6188e4be792




225it [00:01, 206.04it/s]


In [5]:
from swebench.harness.run_evaluation import run_instances
import resource
import docker
from swebench.harness.docker_utils import list_images, clean_images
from swebench.harness.docker_build import build_env_images
from pathlib import Path
import json
import os

RUN_EVALUATION_LOG_DIR = Path("logs/run_evaluation")
LANGSMITH_EVALUATION_DIR = './langsmith_feedback/feedback.json'

def convert_runs_to_langsmith_feedback(
        predictions: dict,
        full_dataset: list,
        run_id: str
    ) -> float:
    """
    Convert logs from docker containers into LangSmith feedback.

    Args:
        predictions (dict): Predictions dict generated by the model
        full_dataset (list): List of all instances
        run_id (str): Run ID
    """
    feedback_for_all_instances = {}

    for instance in full_dataset:
        feedback_for_instance = []
        instance_id = instance['instance_id']
        prediction = predictions[instance_id]
        if prediction.get("model_patch", None) in ["", None]:
            # Prediction returned an empty patch
            feedback_for_all_instances[prediction['run_id']] = [{"key":"non-empty-patch","score":0},
                                                                {"key":"completed-patch","score":0},
                                                                {"key":"resolved-patch","score":0}]
            continue
        feedback_for_instance.append({"key":"non-empty-patch","score":1})
        report_file = (
            RUN_EVALUATION_LOG_DIR
            / run_id
            / prediction["model_name_or_path"].replace("/", "__")
            / prediction['instance_id']
            / "report.json"
        )
        if report_file.exists():
            # If report file exists, then the instance has been run
            feedback_for_instance.append({"key":"completed-patch","score":1})
            report = json.loads(report_file.read_text())
            # Check if instance actually resolved the PR
            if report[instance_id]["resolved"]:
                feedback_for_instance.append({"key":"resolved-patch","score":1})
            else:
                feedback_for_instance.append({"key":"resolved-patch","score":0})
        else:
            # The instance did not run successfully
            feedback_for_instance += [{"key":"completed-patch","score":0},{"key":"resolved-patch","score":0}]
        feedback_for_all_instances[prediction['run_id']] = feedback_for_instance

    os.makedirs(os.path.dirname(LANGSMITH_EVALUATION_DIR), exist_ok=True)
    with open(LANGSMITH_EVALUATION_DIR, 'w') as json_file:
        json.dump(feedback_for_all_instances, json_file)

def evaluate_predictions(
        dataset: list,
        predictions: list,
        max_workers: int,
        force_rebuild: bool,
        cache_level: str,
        clean: bool,
        open_file_limit: int,
        run_id: str,
        timeout: int,
    ):
    """
    Run evaluation harness for the given dataset and predictions.
    """
    # set open file limit
    assert len(run_id) > 0, "Run ID must be provided"
    resource.setrlimit(resource.RLIMIT_NOFILE, (open_file_limit, open_file_limit))
    client = docker.from_env()

    existing_images = list_images(client)
    print(f"Running {len(dataset)} unevaluated instances...")
    # build environment images + run instances
    build_env_images(client, dataset, force_rebuild, max_workers)
    run_instances(predictions, dataset, cache_level, clean, force_rebuild, max_workers, run_id, timeout)

    # clean images + make final report
    clean_images(client, existing_images, cache_level, clean)

    convert_runs_to_langsmith_feedback(predictions,dataset,run_id)

2025-02-11 20:59:53,883 - numexpr.utils - INFO - NumExpr defaulting to 10 threads.


In [7]:
dataset = []
predictions = {}
for res in result:
    predictions[res['run'].outputs['instance_id']] = {**res['run'].outputs,**{"run_id":str(res['run'].id)}}
    dataset.append(res['run'].inputs['inputs'])
for d in dataset:
    d['version'] = d['version'].split(":")[1]

In [ ]:
evaluate_predictions(dataset[:3],predictions,max_workers=8,force_rebuild=False,cache_level="env",clean=False \
                     ,open_file_limit=4096,run_id="test",timeout=1_800)

Running 225 unevaluated instances...
Building base image (sweb.base.py.arm64:latest)
Base images built successfully.
Total environment images to build: 22


  0%|          | 0/22 [00:00<?, ?it/s]

<class 'swebench.harness.docker_build.BuildImageError'>: Error building image sweb.env.py.arm64.e10d470400e85820f705a6:latest: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 137
Check (logs/build_images/env/sweb.env.py.arm64.e10d470400e85820f705a6__latest/build_image.log) for more information.


Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/docker_build.py", line 155, in build_image
    raise docker.errors.BuildError(
docker.errors.BuildError: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 137

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/utils.py", line 92, in run_threadpool
    future.result()
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/Users/matthew.khoriaty/opt/anaconda3

<class 'swebench.harness.docker_build.BuildImageError'>: Error building image sweb.env.py.arm64.7b20504f50e05a8acff8b8:latest: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 137
Check (logs/build_images/env/sweb.env.py.arm64.7b20504f50e05a8acff8b8__latest/build_image.log) for more information.


...

Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/docker_build.py", line 155, in build_image
    raise docker.errors.BuildError(
docker.errors.BuildError: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 1

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/utils.py", line 92, in run_threadpool
    future.result()
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/Users/matthew.khoriaty/opt/anaconda3/e

<class 'swebench.harness.docker_build.BuildImageError'>: Error building image sweb.env.py.arm64.21c198a27f66156c5fdb6e:latest: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 1
Check (logs/build_images/env/sweb.env.py.arm64.21c198a27f66156c5fdb6e__latest/build_image.log) for more information.


Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/docker_build.py", line 155, in build_image
    raise docker.errors.BuildError(
docker.errors.BuildError: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 1

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/utils.py", line 92, in run_threadpool
    future.result()
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/Users/matthew.khoriaty/opt/anaconda3/e

<class 'swebench.harness.docker_build.BuildImageError'>: Error building image sweb.env.py.arm64.29ea89393f6403e072c9f5:latest: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 1
Check (logs/build_images/env/sweb.env.py.arm64.29ea89393f6403e072c9f5__latest/build_image.log) for more information.


Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/docker_build.py", line 155, in build_image
    raise docker.errors.BuildError(
docker.errors.BuildError: The command '/bin/sh -c /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"' returned a non-zero code: 1

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/matthew.khoriaty/Desktop/research/coordination_among_ai/SWE-bench/swebench/harness/utils.py", line 92, in run_threadpool
    future.result()
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.khoriaty/opt/anaconda3/envs/langchain/lib/python3.11/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/Users/matthew.khoriaty/opt/anaconda3/e